## Project 6- Pipeline 2 - Search

In [1]:
import lib.database_module as db
import lib.encoding_module as en
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
transformer = joblib.load('data/vectorizer.pkl')

### Parse: Encode search as document vector	

In [3]:
search_string = 'lamb spice'
search_vector = en.get_searchterm_vector(transformer,search_string)
search_vector

{'lamb spice': array([[  9.39196867e-04,   1.43169565e-03,  -1.21168595e-03,
           3.61385535e-03,   4.74564896e-03,  -5.80760501e-04,
           1.49747297e-03,  -7.23857591e-05,   1.10184292e-04,
           9.85509996e-04,   1.62148492e-04,  -5.26374090e-04,
           2.13772876e-04,  -6.20850252e-04,   2.34898152e-04,
           1.51108260e-04,  -3.92029581e-04,  -2.57289221e-04,
           6.71083763e-04,   1.64903836e-04,   7.65058185e-05,
          -1.17173696e-04,   1.54970421e-04,  -8.10201235e-04,
           1.09032702e-03,   4.68790357e-05,  -1.24313210e-03,
           2.32990425e-04,   4.73121052e-04,  -9.73913550e-04,
           1.10966773e-03,   7.94250375e-04,  -7.07979862e-04,
          -3.54431421e-04,   2.87902676e-03,   6.27188241e-04,
          -5.98395240e-03,  -3.56760281e-03,  -6.53397680e-03,
           7.67086702e-04,  -4.27455173e-03,  -1.94911021e-03,
           2.08119002e-03,  -1.23431164e-02,   1.12193464e-02,
          -7.15277372e-03,   8.66292486e-

### Mine: Select document vectors for all pages from database

In [4]:
pages_vectors = db.select_all_page_vectors(location='local_group_solution')
len(pages_vectors)

Connected to server localhost.


122

### Model: Find five most similar documents based on document vectors

In [5]:
from sklearn.neighbors import NearestNeighbors


In [6]:
search_vector[search_vector.keys()[0]]

array([[  9.39196867e-04,   1.43169565e-03,  -1.21168595e-03,
          3.61385535e-03,   4.74564896e-03,  -5.80760501e-04,
          1.49747297e-03,  -7.23857591e-05,   1.10184292e-04,
          9.85509996e-04,   1.62148492e-04,  -5.26374090e-04,
          2.13772876e-04,  -6.20850252e-04,   2.34898152e-04,
          1.51108260e-04,  -3.92029581e-04,  -2.57289221e-04,
          6.71083763e-04,   1.64903836e-04,   7.65058185e-05,
         -1.17173696e-04,   1.54970421e-04,  -8.10201235e-04,
          1.09032702e-03,   4.68790357e-05,  -1.24313210e-03,
          2.32990425e-04,   4.73121052e-04,  -9.73913550e-04,
          1.10966773e-03,   7.94250375e-04,  -7.07979862e-04,
         -3.54431421e-04,   2.87902676e-03,   6.27188241e-04,
         -5.98395240e-03,  -3.56760281e-03,  -6.53397680e-03,
          7.67086702e-04,  -4.27455173e-03,  -1.94911021e-03,
          2.08119002e-03,  -1.23431164e-02,   1.12193464e-02,
         -7.15277372e-03,   8.66292486e-03,  -2.02755677e-03,
        

In [7]:
vectors = [x[1] for x in pages_vectors]
page_ids = [x[0] for x in pages_vectors]

nn= NearestNeighbors(algorithm='kd_tree', radius =0.000001 )
nn.fit(vectors, page_ids)


distance, indices= nn.kneighbors(search_vector[search_vector.keys()[0]],n_neighbors=5)
page_match_list = [page_ids[i] for i in list(indices[0])]


In [8]:
for snip in db.select_pages(page_match_list,'local_group_solution'):
    print snip[2][:250]+'...\n'

Connected to server localhost.
Dabeli or Kutchi dabeli or Double Roti (Devnagari: दाबेली - कच्छी दाबेली) is a snack food of India, originating in the Kutch region of Gujarat. It is a spicy snack made by mixing boiled potatoes with a dabeli masala,...

Doubles is a common street food in Trinidad and Tobago. It is a sandwich made with two baras (flat fried bread) filled with curry channa (curried chick peas). See also Cuisine of Trinidad and Tobago References External links The Origin of Trinidad Do...

Bake and Shark is a traditional fast food dish of Trinidadian cuisine. Preparation Bake and Shark is a classic street food dish that is sold at a multitude of food stalls and cookshops all over Trinidad and Tobago. It consists of a fried flatbread ("...

Vada pav (Marathi: वडा पाव), sometimes spelled wada pav or vada paav or vada pao or wada pao, is a vegetarian fast food dish native to the Indian state of Maharashtra. The dish is a simple creation involving a deep fried potato patty with

In [9]:
cos = cosine_similarity(vectors,search_vector[search_vector.keys()[0]])

In [10]:
indices = cos.argsort(axis = 0)[-5:].ravel().tolist()
indices.reverse()
page_match_list = [page_ids[i] for i in indices]
print page_match_list

[341075, 877704, 453166, 2546911, 9074556]


In [11]:
temp_list =[]
for page in page_match_list:
    connection, cur = db.connect_to_postgres('local')
    sql = """SELECT title, page
             FROM page
             WHERE page_id = {};""".format(page)
    cur.execute(sql)
    temp = cur.fetchone()
    #print "Page title: {}\n{}\n\n".format(temp[0], temp[1][:250])
    temp_list.append( (temp[0], temp[1][:250]))
    connection.commit()
    cur.close()
    connection.close()
    
for result in temp_list:
    print "Page title: {}\n{}\n\n".format(result[0], result[1][:250])

Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Page title: Gyro (food)
A gyro or gyros (Greek pronunciation: [ˈYEEros]) (Greek: γύρος, gyros, literally turn) is a Greek dish made of meat cooked on a vertical rotisserie, traditionally pork, or chicken, and outside of Greece with beef, veal, lamb, and usually serve


Page title: Doner kebab
Döner kebab (/ˈdɒnər kəˈbæb/, /ˈdoʊnər/ Turkish: döner or döner kebap, [døˈnɛɾ ˈcebɑp], in English often spelled doner or donner) is a type of Turkish kebab, made of meat cooked on a vertical rotisserie. Similar dishes cooked on a v


Page title: Shawarma
Shawarma or Shawurma (Arabic: شاورما‎‎ / ALA-LC: shāwarmā Hebrew: שַׁוַארְמָה‎ shawarmah) is a Levantine meat preparation, where lamb, chicken, turkey, beef, veal, buffalo meat, or mixed meats are placed on a spit (commonly a


Page title: Al pastor
Al pastor (from Spanish, lit. In the style of

In [1]:
string = 'list\nlist\n'


In [2]:
string.replace("\n"," ")

'list list '